In [1]:
import os
import glob

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic' # font 설정
matplotlib.rcParams['axes.unicode_minus'] = False # '-' 부호 인식 설정

import seaborn as sns

In [2]:
obs_data_list = glob.glob("../DATA/20230705_스시2_서울&시흥_1월관측_후처리자료/*csv") # observational data file list

# 바이살라 수집 데이터 분석

In [3]:
obs_data_list_vi = glob.glob("../DATA/20230705_스시2_서울&시흥_1월관측_후처리자료/*vai*csv") # observational data file list

In [4]:
obs_data_list_lufft = glob.glob("../DATA/20230705_스시2_서울&시흥_1월관측_후처리자료/*lufft*csv") # observational data file list

바이살라 데이터의 컬럼이름은 모두 일치하는 것을 확인

In [5]:
gangbyeon_vaisala_20230119_U_df = pd.read_csv(obs_data_list_vi[0])
naebu_vaisala_20230119_U_df = pd.read_csv(obs_data_list_vi[1])
dongbu_vaisala_20230119_U_df = pd.read_csv(obs_data_list_vi[2])

In [6]:
gangbyeon_vaisala_20230119_U_df['timestamp'] = pd.to_datetime(gangbyeon_vaisala_20230119_U_df['timestamp'])

# 관측 데이터와 모델 데이터 병합 바이살라

In [7]:
from sklearn.neighbors import KDTree
from sklearn.metrics import mean_squared_error

In [ ]:
# extract observational data information from file name
def load_vi_data(file_name):
    # file_name = obs_data_list[0] # select one file from file list
    obs_df = pd.read_csv(file_name,usecols = ['timestamp','longitude','latitude','surface_temperature'])
    obs_df['timestamp'] = pd.to_datetime(obs_df['timestamp'])
    obs_df['day'] = obs_df['timestamp'].dt.day
    obs_df['hour'] = obs_df['timestamp'].dt.hour
    file_info = file_name.split("\\")[-1].split("_") # split the name for information
    obs_site = file_info[0] #  get observation site information
    sensor =  file_info[-3] 
    obs_road,obs_road_direction= file_info[-4], file_info[-1].split(".")[0]
    year_month, day = file_info[-2][:6], file_info[-2][6:8] # get time imformation

    return obs_df, obs_site, obs_road, obs_road_direction, year_month, day, sensor

# extract observational data information from file name
def load_lufft_data(file_name):
    # file_name = obs_data_list[0] # select one file from file list
    obs_df = pd.read_csv(file_name,usecols = ['Timestamp','longitude','latitude','Road temperature100 [°C] Cur'])
    obs_df = obs_df.rename({'Road temperature100 [°C] Cur':'surface_temperature'},axis=1)
    obs_df.columns = obs_df.columns.str.lower()
    obs_df['timestamp'] = pd.to_datetime(obs_df['timestamp'])
    obs_df['day'] = obs_df['timestamp'].dt.day
    obs_df['hour'] = obs_df['timestamp'].dt.hour
    file_info = file_name.split("\\")[-1].split("_") # split the name for information
    obs_site = file_info[0] #  get observation site information
    sensor =  file_info[-3] 
    obs_road,obs_road_direction= file_info[-4], file_info[-1].split(".")[0]
    year_month, day = file_info[-2][:6], file_info[-2][6:8] # get time imformation

    return obs_df, obs_site, obs_road, obs_road_direction, year_month, day, sensor

def load_model_data(obs_site, year_month, day, obs_road, obs_road_direction):
    model_df_all_site_list = []
    model_sites = ['jr','mg','org','ss','yc']
    for model_site in model_sites:
        try:
            model_data_list = [glob.glob(f"../DATA/MODEL/{obs_site}/{model_site}/{year_month}/{day}/*csv")[0],glob.glob(f"../DATA/MODEL/{obs_site}/{model_site}/{year_month}/{int(day)-1}/*csv")[0]]
        except:
            model_data_list = [glob.glob(f"../DATA/MODEL/siheung/{model_site}/{year_month}/{day}/*csv")[0],glob.glob(f"../DATA/MODEL/siheung/{model_site}/{year_month}/{int(day)-1}/*csv")[0]]
        model_df_list = []
        for model_file in model_data_list:
            model_df_indiv =  pd.read_csv(model_file,usecols = ['date_time','update_time','lon','lat','road_name','direction','road_temp','p_hour'])
            cond1 = model_df_indiv['road_name'] == obs_road
            cond2 = model_df_indiv['direction'].str.startswith(obs_road_direction)
            model_df_indiv = model_df_indiv[cond1 & cond2]
            model_df_list.append(model_df_indiv)
        model_df = pd.concat(model_df_list)

        model_df['date_time'] = pd.to_datetime(model_df['date_time'])
        model_df = model_df[model_df['date_time'].dt.day == int(day)]
        model_df = model_df[~((model_df['date_time'].dt.hour == 15) & (model_df['p_hour']==24))]
        model_df = model_df.rename({"road_temp":f"{model_site}"}, axis =1)

        model_df_all_site_list.append(model_df)

    model_data_df = model_df_all_site_list[0]
    for df in model_df_all_site_list[1:]:
        model_data_df = pd.merge(model_data_df, df, on=['lon', 'lat', 'date_time', 'update_time', 'p_hour', 'road_name', 'direction'])
    model_data_df['date_time'] = pd.to_datetime(model_data_df['date_time'])
    model_data_df['hour'] = model_data_df['date_time'].dt.hour
    model_data_df['day'] = model_data_df['date_time'].dt.day

    return model_data_df

def perform_kdtree_matching_vi(obs_df, model_data_df):
    # 첫 번째 줄의 위경도 데이터
    observ_line = np.array(list(zip(obs_df['longitude'], obs_df['latitude'])))

    # 두 번째 줄의 위경도 데이터
    model_line = np.array(list(zip(model_data_df['lon'], model_data_df['lat'])))

    # KDTree 객체 생성
    tree = KDTree(observ_line)

    # 각 점마다 가장 가까운 점을 찾아 매칭
    matched_points = []
    for point in model_line:
        _, index = tree.query([point], k=1)  # k=1로 설정하여 가장 가까운 점 하나만 선택
        matched_points.append(observ_line[index[0]])

    matched_lon = [point[0][0] for point in matched_points]
    matched_lat = [point[0][1] for point in matched_points]

    df = pd.DataFrame({'lon': model_line[:, 0], 'lat': model_line[:, 1],
                       'longitude': matched_lon, 'latitude': matched_lat})

    total_df = pd.merge(df, obs_df, on=['longitude', 'latitude'])
    total_df = pd.merge(total_df, model_data_df, on=['lon', 'lat','hour','day'])
    
    total_df = total_df.drop_duplicates()

    return total_df

def perform_kdtree_matching_lufft(obs_df, model_data_df):
    # 첫 번째 줄의 위경도 데이터
    observ_line = np.array(list(zip(obs_df['longitude'], obs_df['latitude'])))

    # 두 번째 줄의 위경도 데이터
    model_line = np.array(list(zip(model_data_df['lon'], model_data_df['lat'])))

    # KDTree 객체 생성
    tree = KDTree(model_line)

    # 각 점마다 가장 가까운 점을 찾아 매칭
    matched_points = []
    for point in observ_line:
        _, index = tree.query([point], k=1)  # k=1로 설정하여 가장 가까운 점 하나만 선택
        matched_points.append(model_line[index[0]])

    matched_lon = [point[0][0] for point in matched_points]
    matched_lat = [point[0][1] for point in matched_points]

    df = pd.DataFrame({'longitude': observ_line[:, 0], 'latitude': observ_line[:, 1],
                       'lon': matched_lon, 'lat': matched_lat})

    total_df = pd.merge(df, obs_df, on=['longitude', 'latitude'])
    total_df = pd.merge(total_df, model_data_df, on=['lon', 'lat','hour','day'])
    
    total_df = total_df.drop_duplicates()

    return total_df


# 각 행에 대해 가장 가까운 값을 찾는 함수
def find_closest(row):
    temp = row['surface_temperature']
    values = row[['jr', 'mg', 'org', 'ss', 'yc']].dropna() 
    closest_val = values.sub(temp).abs().idxmin()
    return pd.Series([values[closest_val], closest_val])

def main_process_vi(file):
    obs_df, obs_site, obs_road, obs_road_direction, year_month, day, sensor = load_vi_data(file)
    model_data_df = load_model_data(obs_site, year_month, day, obs_road, obs_road_direction)
    df = perform_kdtree_matching_vi(obs_df,model_data_df)

    # 새로운 컬럼을 추가하여 결과 저장
    df_1 = df.reset_index()
    df_1 = df_1.drop(['index'],axis = 1)
    df_1[['closest_value', 'closest_column']] = df_1.apply(find_closest, axis=1)
    df_1.to_csv(f"../OUTPUT/{obs_site}_{obs_road}_{obs_road_direction}_{year_month}_{day}_{sensor}_concat.csv", index=None)

def main_process_lufft(file):
    obs_df, obs_site, obs_road, obs_road_direction, year_month, day, sensor = load_lufft_data(file)
    model_data_df = load_model_data(obs_site, year_month, day, obs_road, obs_road_direction)
    df = perform_kdtree_matching_lufft(obs_df,model_data_df)

    # 새로운 컬럼을 추가하여 결과 저장
    df_1 = df.reset_index()
    df_1 = df_1.drop(['index'],axis = 1)
    df_1[['closest_value', 'closest_column']] = df_1.apply(find_closest, axis=1)
    df_1.to_csv(f"../OUTPUT/{obs_site}_{obs_road}_{obs_road_direction}_{year_month}_{day}_{sensor}_concat.csv", index=None)


obs_data_list = glob.glob("../DATA/20230705_스시2_서울&시흥_1월관측_후처리자료/*csv") # observational data file list
obs_data_list_vi = glob.glob("../DATA/20230705_스시2_서울&시흥_1월관측_후처리자료/*vai*csv") # observational data file list
obs_data_list_lufft = glob.glob("../DATA/20230705_스시2_서울&시흥_1월관측_후처리자료/*lufft*csv") # observational data file list


In [18]:
for i in obs_data_list_vi:
    main_process_vi(i)

for i in obs_data_list_lufft:
    main_process_lufft(i)

# RMSE계산

In [19]:
from sklearn.metrics import mean_squared_error
import numpy as np
df_list = glob.glob('C:/Users/user/Desktop/모델검증/output/S*.csv')
# 두 개의 컬럼을 가진 데이터 프레임 df가 있다고 가정합니다.
# 'actual'과 'predicted'라는 두 개의 컬럼이 있다고 가정합니다.
for j in df_list:
    df = pd.read_csv(j)
    # 실제값과 예측값을 가져옵니다.
    print(j)
    for i in ['org', 'jr',  'ss', 'yc','mg','closest_value']:
        actual = df['surface_temperature']
        predicted = df[i]

        # MSE(Mean Squared Error)를 계산합니다.
        mse = mean_squared_error(actual, predicted)

        # MSE의 제곱근을 계산하여 RMSE를 얻습니다.
        rmse = np.sqrt(mse)

        print(f"{i} RMSE:", rmse, end = "\t")
    print()

C:/Users/user/Desktop/모델검증/output\seoul_dongbu_D_202301_26_lufft_concat.csv
org RMSE: 3.802412704758773	jr RMSE: 4.566446093405256	ss RMSE: 0.9559310086577861	yc RMSE: 0.8912229052611237	mg RMSE: 4.379045306558021	closest_value RMSE: 0.8908008170118288	
C:/Users/user/Desktop/모델검증/output\seoul_dongbu_D_202301_27_lufft_concat.csv
org RMSE: 2.509569535865475	jr RMSE: 2.032480403011926	ss RMSE: 5.099827972368705	yc RMSE: 5.035258856934394	mg RMSE: 1.9843988981435567	closest_value RMSE: 0.7127833900026213	
C:/Users/user/Desktop/모델검증/output\seoul_dongbu_U_202301_19_vaisala_concat.csv
org RMSE: 1.7172232215597192	jr RMSE: 1.934114923990452	ss RMSE: 4.2887247559282	yc RMSE: 3.811792209419844	mg RMSE: 3.3797400977365095	closest_value RMSE: 0.7882728567165652	
C:/Users/user/Desktop/모델검증/output\seoul_gangbyeon_U_202301_19_vaisala_concat.csv
org RMSE: 1.4231696896452413	jr RMSE: 4.436327548398261	ss RMSE: 0.9463336079678354	yc RMSE: 1.33121238944142	mg RMSE: 7.195495881405455	closest_value RMSE: 0